# Train-Test Split of heating data

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

In [5]:
#use csv with already preprocessed features (scaling, differnecing etc needed)
df = pd.read_csv("somewhatcleaned_heatingdemand.csv", index_col=0)

### some small data checks

In [6]:
df

,total_heat
2007-12-31 22:00:00+00:00,95296.0
2007-12-31 23:00:00+00:00,95296.0
2008-01-01 00:00:00+00:00,98086.0
2008-01-01 01:00:00+00:00,100454.0
2008-01-01 02:00:00+00:00,107183.0
...,...
2015-01-01 19:00:00+00:00,82364.0
2015-01-01 20:00:00+00:00,74662.0
2015-01-01 21:00:00+00:00,60137.0
2015-01-01 22:00:00+00:00,49587.0


In [8]:
df.index = pd.DatetimeIndex(df.index)

In [19]:
df.groupby(df.index.floor('d')).count().value_counts()

total_heat
24            2558
2                1
dtype: int64

In [33]:
# removing the 2 values from first year if necessary
np_array = df.iloc[2:].to_numpy()

## Creating input X and target y

In [12]:
# Hyperparameter
train_days = 10
forecast_days = 2

In [40]:
len(np_array[train_days*24:-forecast_days*24])

61104

Possibility 1

In [73]:
# creating tuples with x and y
arr = [
    (np_array[i-train_days*24:i], np_array[i:i+forecast_days*24]) for 
    i  in range(train_days*24, len(np_array)-forecast_days*24)
    ]

Possibility 2

In [74]:
# creating two seperate lists with a function

def create_X_y(data_array: np.array, train_days:int, forecast_days:int):
    """"""
    X_list = []
    y_list = []

    for i in range(train_days*24, len(data_array)-forecast_days*24):
        X_list.append(data_array[i-train_days*24:i])
        y_list.append(data_array[i:i+forecast_days*24])
    
    return X_list, y_list
    

In [76]:
X_list, y_list = create_X_y(np_array, train_days, forecast_days)

In [77]:
len(X_list), len(y_list)

(61104, 61104)

## Data split in train, eval, test

Should decide that on a portion of data which is not for hyperparam tuning, but is only used at the very end.  (Test-data)
-> Maybe last year of data

Could decide that one portion of data will be used for hyperparam tuning, but not for training of data. (eval-data)
-> Maybe second last year

In [30]:
np_test = np_array[-365*24:]
np_eval = np_array[-365*24*2:-365*24]
np_train = np_array[:-365*24*2]
len(np_test), len(np_eval), len(np_train)

(8760, 8760, 43874)